In [ ]:
#CNN 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# Load the dataset
data = pd.read_csv('AAPL.csv')
df = data.drop('Date', axis=1)

# Assuming 'Close' is the target and all other columns are features
X = df.drop('Adj Close', axis=1).values
y = df['Adj Close'].values.reshape(-1,1)

# Scale the features and target
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape input for the CNN [samples, time_steps, features]
time_steps = 1  # Example time steps
X_train_cnn = X_train.reshape((X_train.shape[0], time_steps, X_train.shape[1]))
X_test_cnn = X_test.reshape((X_test.shape[0], time_steps, X_test.shape[1]))

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(time_steps, X_train.shape[1])))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train_cnn, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Predicting the Test set results
y_pred_scaled = model.predict(X_test_cnn)

# Calculating RMSE and MAE using the scaled data
mae_scaled = mean_absolute_error(y_test, y_pred_scaled)
rmse_scaled = np.sqrt(mean_squared_error(y_test, y_pred_scaled))

print(f'Scaled MAE: {mae_scaled}')
print(f'Scaled RMSE: {rmse_scaled}')


In [ ]:
#BiLSTM 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('AAPL.csv')

# Preprocessing
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')
data.set_index('Date', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
data_normalized = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

# Define a scaler for the target column if separate scaling is needed
scaler_target = MinMaxScaler()
data_normalized['Adj Close'] = scaler_target.fit_transform(data[['Close']])

# Create the sequences
lookback = 5
X, Y = [], []
for i in range(lookback, len(data_normalized)):
    X.append(data_normalized.iloc[i-lookback:i].values)
    Y.append(data_normalized.iloc[i]['Close'])
X, Y = np.array(X), np.array(Y)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
trainX, testX = X[:train_size], X[train_size:]
trainY, testY = Y[:train_size], Y[train_size:]

# Reshape the input for LSTM compatibility
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], data_normalized.shape[1]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], data_normalized.shape[1]))

# Define the BiLSTM model
model = Sequential()

# Add a Bidirectional LSTM layer
model.add(Bidirectional(LSTM(50, return_sequences=True, activation='relu'), input_shape=(lookback, data_normalized.shape[1])))
model.add(Bidirectional(LSTM(50, activation='relu')))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(testX, testY), verbose=1)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Predict on the test data
preds = model.predict(testX)

# Calculate and print MSE and MAE using the scaled predictions and actual values
mse_scaled = mean_squared_error(testY, preds)
mae_scaled = mean_absolute_error(testY, preds)
rmse_scaled = np.sqrt(mse_scaled)

print(f'Scaled MSE: {mse_scaled}, Scaled MAE: {mae_scaled}, Scaled RMSE: {rmse_scaled}')

In [ ]:
#LSTM
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Set random seed for reproducibility
np.random.seed(42)

# Load the data
data = pd.read_csv('AAPL.csv') #change the dataset here 

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1)).flatten() # Assuming 'Close' is the column you want to predict

# Data Preprocessing Function modified for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 1  # Updated sequence length for your use case
steps = 1  # Number of future steps to predict
X, y = create_sequences(normalized_data, sequence_length, steps)

# Split the data
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))
model.add(Dense(steps))  # The output layer should have 'steps' units if predicting multiple steps

# Compile the model
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(X_test, y_test))

# Predicting
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Evaluate the model
mae_train = mean_absolute_error(y_train, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predict))

mae_test = mean_absolute_error(y_test, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test, test_predict))

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")

# If you want to plot the predictions (optional)
import matplotlib.pyplot as plt
plt.plot(y_test)
plt.plot(test_predict)
plt.show()


In [ ]:
#SVR
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torch.optim.lr_scheduler import OneCycleLR
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Load the data
data_path = 'AAPL.csv'  # Update this to the correct path where lorenz.txt is located
data = np.loadtxt(data_path)

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data.reshape(-1, 1)).flatten()

# Data Preprocessing Function modified for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 1  
steps = 1  # Number of future steps to predict
X, y = create_sequences(normalized_data, sequence_length, steps)

# Split the data
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Instantiate and fit the SVR model
# Note: You may want to tune these hyperparameters
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_rbf.fit(X_train, y_train.ravel())

# Predictions
train_predict = svr_rbf.predict(X_train)
test_predict = svr_rbf.predict(X_test)

# Evaluate the model using MAE and RMSE on the scaled data
mae_train = mean_absolute_error(y_train, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predict))

mae_test = mean_absolute_error(y_test, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test, test_predict))

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")


In [ ]:
#GRU
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torch.optim.lr_scheduler import OneCycleLR
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Load the data
data_path = 'AAPL.csv'  # Update this to the correct path where lorenz.txt is located
data = np.loadtxt(data_path)

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data.reshape(-1, 1)).flatten()

# Data Preprocessing Function modified for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 1  
steps = 1  # Number of future steps to predict
X, y = create_sequences(normalized_data, sequence_length, steps)

# Convert to PyTorch tensors
X_train_tensor = torch.Tensor(X_train).unsqueeze(-1)  # Add channel dimension
X_test_tensor = torch.Tensor(X_test).unsqueeze(-1)  # Add channel dimension
y_train_tensor = torch.Tensor(y_train)
y_test_tensor = torch.Tensor(y_test)

# Create DataLoader instances
batch_size = 64  # You can adjust the batch size
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

# Define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Instantiate the model
input_size = 1  # Number of features
hidden_size = 50  # Number of features in hidden state
num_layers = 2  # Number of stacked GRU layers
output_size = 1  # Number of output predictions

model = GRUModel(input_size, hidden_size, num_layers, output_size)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100  # You can adjust the number of epochs

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).numpy()
    mae_test = mean_absolute_error(y_test, predictions)
    rmse_test = np.sqrt(mean_squared_error(y_test, predictions))

print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")